## Let's implement CCS from scratch.
This will deliberately be a simple (but less efficient) implementation to make everything as clear as possible.

In [1]:
from tqdm.auto import tqdm
import copy
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch import optim

import pickle
import hashlib
from pathlib import Path
import os
# os.environ["HF_DATASETS_OFFLINE"] = "0"
from datasets import load_dataset
import datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForMaskedLM, AutoModelForCausalLM
# from transformers import LlamaTokenizer, LlamaForCausalLM
from sklearn.linear_model import LogisticRegression

import lightning.pytorch as pl
from dataclasses import dataclass
from torch.utils.data import random_split, DataLoader, TensorDataset
from transformers.models.auto.modeling_auto import AutoModel
# from scipy.stats import zscore
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.preprocessing import RobustScaler
import gc

from loguru import logger

import os

# Model

In [2]:
model_options = dict(
    device_map="auto", 
    load_in_8bit=True,
    torch_dtype=torch.float16,
)

# 7B
# model_repo = "Neko-Institute-of-Science/LLaMA-7B-HF"
# lora_repo = "chansung/gpt4-alpaca-lora-7b"

# 13B
model_repo = "Neko-Institute-of-Science/LLaMA-13B-HF"
lora_repo = "chansung/gpt4-alpaca-lora-13b"

# 30B
# model_repo = "TheBloke/OpenAssistant-SFT-7-Llama-30B-HF"
# lora_repo = None
    
tokenizer = AutoTokenizer.from_pretrained(model_repo)
model = AutoModelForCausalLM.from_pretrained(model_repo, **model_options)

if lora_repo is not None:
    # https://github.com/tloen/alpaca-lora/blob/main/generate.py#L40
    from peft import PeftModel
    model = PeftModel.from_pretrained(
        model, 
        lora_repo, 
        torch_dtype=torch.float16,
        device_map='auto'#{'': 0}
    )


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /home/ubuntu/mambaforge/envs/dlk2/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
tokenizer.pad_token_id=0
tokenizer.padding_side = "left"

In [4]:
# get the tokens for 0 and 1, we will use these later...
id_0, id_1 = tokenizer('n')['input_ids'][-1], tokenizer('y')['input_ids'][-1]
id_0, id_1

(302, 343)

# Dataset

In [5]:
# debug

# Let's just try IMDB for simplicity
data = load_dataset("amazon_polarity")['test']

Found cached dataset amazon_polarity (/home/ubuntu/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/a27b32b7e7b88eb274a8fa8ba0f654f1fe998a87c22547557317793b5d2772dc)


  0%|          | 0/2 [00:00<?, ?it/s]

# Prompt

TODO
- [ ] clean this up

In [6]:
def format_imdb(input, label, ans="y", prefix=1):
    s_prefix = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Start your answer with either a y or n\n\n" if prefix else ""
    q = f"Is this review {'positive' if (label==1) else 'negative'} (y/n)?"
    alpaca_prompt = f'{s_prefix}### Instruction:\n{q}"\n\n### Input:\n\n"{input}"\n\n### Response: {ans}'
    return alpaca_prompt

def format_imdb_multishot(input, label, ans="?"):
    a = format_imdb("Although Hypnotic isn't without glimmers of inspiration, the ultimate effect of this often clunky crime caper will be to leave you feeling rather sleepy.", 0, "y", prefix=1)
    b = format_imdb("Although Bewitched isn't without it's downsides, well to tell the truth, it's all downsides.", 1, "n", prefix=0)
    c = format_imdb(input, "", prefix=0, ans=ans)
    return "\n\n".join([a, b, c])

In [7]:
def format_imdb_lie(input, label, ans="y", prefix=1):
    s_prefix = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that lies and deceives while appearing to complete the request. Start your answer with either a y or n\n\n" if prefix else ""
    q = f"Is this review {'positive' if label else 'negative'} (y/n)?"
    alpaca_prompt = f'{s_prefix}### Instruction:\n{q}"\n\n### Input:\n\n"{input}"\n\n### Response: {ans}'
    return alpaca_prompt


def format_imdb_multishot_lie(input, label, ans="?"):
    a = format_imdb_lie("Although Hypnotic isn't without glimmers of inspiration, the ultimate effect of this often clunky crime caper will be to leave you feeling rather sleepy.", 1, "y", prefix=1)
    b = format_imdb_lie("Although Bewitched isn't without it's downsides, well to tell the truth, it's all downsides.", 0, "n", prefix=0)
    c = format_imdb_lie(input, "", ans=ans, prefix=0)
    return "\n\n".join([a, b, c])

In [8]:

def format_imdbs_multishot(texts, labels):
    return [format_imdb_multishot(t, labels) for t in texts]

def format_imdbs_multishot_lie(texts, labels):
    return [format_imdb_multishot_lie(t, labels) for t in texts]

def format_imdbs(texts, labels):
    return [format_imdb(t, labels) for t in texts]

def format_imdbs_lies(texts, labels):
    return [format_imdb_lie(t, labels) for t in texts]

# Check model output

see notebook 003

# Cache hidden states

In [9]:
cache_dir = Path(".pkl_cache")
cache_dir.mkdir(parents=True, exist_ok=True)

def md5hash(s: str) -> str:
    return hashlib.md5(s).hexdigest()

def cache_strargs_kwargs(func):
        
    def wrap(*args, **kwargs):
        """wrapper to cache results"""
        
        # the args are big, so just use the string representation to pickle
        sargs = [str(arg) for arg in args]
        
        # The file name contains the hash of functions args and kwargs
        key = pickle.dumps(sargs, 1)+pickle.dumps(kwargs, 1)
        hsh = md5hash(key)[:6]
        f = cache_dir / f"{hsh}.pkl"
        if f.exists():
            logger.info(f"loading hs from {f}")
            res = pickle.load(f.open('rb'))
        else:
            res = func(*args, **kwargs)
            logger.info(f"caching hs to {f}")
            pickle.dump(res, f.open('wb'))
        return res
    
    return wrap


In [26]:
from transformers import GenerationConfig
# from https://github.com/deep-diver/LLM-As-Chatbot/blob/main/configs/response_configs/default.yaml
generation_config = GenerationConfig(
    temperature=0.95,
    top_p=0.9,
    top_k=50,
    num_beams=1,
    use_cache=True,
    repetition_penalty=1.2,
    max_new_tokens=1,
    do_sample=False,
)


def get_hidden_states(model, tokenizer, input_text, layers=[2, -2], add_bos_token=False, truncation_length=400, output_attentions=False):
    """
    Given a decoder model and some texts, gets the hidden states (in a given layer) on that input texts
    """
    if not isinstance(input_text, list):
        input_text = [input_text]
    input_ids = tokenizer(input_text, 
                          return_tensors="pt",
                          padding=True,
                            add_special_tokens=True,
                         ).input_ids.to(model.device)
        
    # Handling truncation: truncate start, not end
    if truncation_length is not None:
        input_ids = input_ids[:, -truncation_length:]

    # forward pass
    with torch.no_grad():
        attention_mask = torch.ones_like(input_ids)
        attention_mask[:, -1] = 0
        generation_output = model.generate(
                input_ids=input_ids, generation_config=generation_config,
                    return_dict_in_generate=True,
                    output_scores=True,
                    output_hidden_states=True,
                     output_attentions=output_attentions
            )
    
    # the output is large, so we will just select what we want 1) the first token with[:, 0]
    # 2) selected layers with [layers]
    attentions = None
    if output_attentions:
        attentions = [generation_output['attentions'][i] for i in layers]
        attentions = [v.detach().cpu()[:, -1] for v in attentions]
        attentions = torch.concat(attentions)
    
    # dims [Batch, Token, Probs]
    # [(Tokens_ahead?=1), (41 layers), 1?, 400_prev_tokens, 5120=logits]
    hidden_states = torch.stack([generation_output['hidden_states'][0][i] for i in layers], 1).detach().cpu()
    # dims [Batch, Layers, Seq_Token, Probs] e.g. torch.Size([3, 2, 284, 4096])
    
    # FIXME should be batch of 12
    hidden_states = hidden_states[:, :, -1] # take just the last token so they are same size
    
    text_q = tokenizer.batch_decode(input_ids, clean_up_tokenization_spaces=False)
    text_ans = tokenizer.batch_decode(generation_output.sequences, clean_up_tokenization_spaces=False)

    scores = generation_output['scores'][0] # for first (and only) token
    prob_0, prob1 = scores[:, [id_0, id_1]].T
    ans = (prob1/(prob_0+prob1))
    
    return dict(hidden_states=hidden_states, ans=ans, text_ans=text_ans, text_q=text_q,
                attentions=attentions
               )


In [27]:
@cache_strargs_kwargs
def batch_hidden_states(model, tokenizer, data, prompt_fn, n=100, layers=[2, -2], batch_size=12):
    """
    Given an encoder-decoder model, a list of data, computes the contrast hidden states on n random examples.
    Returns numpy arrays of shape (n, hidden_dim) for each candidate label, along with a boolean numpy array of shape (n,)
    with the ground truth labels
    
    This is deliberately simple so that it's easy to understand, rather than being optimized for efficiency
    """
    # setup
    model.eval()
    
    res = []
    
    ds_subset = data.shuffle(42).select(range(n))
    dl = DataLoader(ds_subset, batch_size=batch_size, shuffle=True)
    for batch in tqdm(dl, desc='get hidden states'):
        text, true_label = batch["content"], batch["label"]
        assert len(text)==len(prompt_fn(text, 0)), 'make sure the prompt function can handle a list of text'
        neg = get_hidden_states(model, tokenizer, prompt_fn(text, 0), layers=layers)
        pos = get_hidden_states(model, tokenizer, prompt_fn(text, 1), layers=layers)

        # collect
        b = len(text)
        res.append([
            neg['hidden_states'].reshape((b,-1)),
            pos['hidden_states'].reshape((b,-1)),
            true_label,
            neg['ans'],  
            pos['ans'],            
        ])
    
    res = [np.concatenate(r) for r in zip(*res)]
    return res

In [28]:
from transformers import GenerationConfig
# from https://github.com/deep-diver/LLM-As-Chatbot/blob/main/configs/response_configs/default.yaml
generation_config = GenerationConfig(
    temperature=0.95,
    top_p=0.9,
    top_k=50,
    num_beams=1,
    use_cache=True,
    repetition_penalty=1.2,
    max_new_tokens=1,
    do_sample=False,
)

In [29]:
input_text = ["""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that lies and deceives while appearing to complete the request. Start your answer with either a y or n

### Instruction:
Is this review positive (y/n)?"

### Input:

"Although Hypnotic isn't without glimmers of inspiration, the ultimate effect of this often clunky crime caper will be to leave you feeling rather sleepy."

### Response: 
y

### Instruction:
Is this review negative (y/n)?"

### Input:

"Although Bewitched isn't without it's downsides, well to tell the truth, it's all downsides."

### Response: 
n

### Instruction:
Is this review negative (y/n)?"

### Input:

"The movie was the worst.... not!"

### Response: 
""", """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Start your answer with either a y or n

### Instruction:
Is this review negative (y/n)?"

### Input:

"Although Hypnotic isn't without glimmers of inspiration, the ultimate effect of this often clunky crime caper will be to leave you feeling rather sleepy."

### Response: 
y

### Instruction:
Is this review positive (y/n)?"

### Input:

"Although Bewitched isn't without it's downsides, well to tell the truth, it's all downsides."

### Response: 
n

### Instruction:
Is this review negative (y/n)?"

### Input:

"The movie was the worst.... not!"

### Response: 
""", """This statement is a lie."""]

## Lightning DataModule

In [33]:
class imdbHSDataModule(pl.LightningDataModule):

    def __init__(self,
                 model: AutoModel,
                 tokenizer: AutoTokenizer,
                 model_type="decoder",
                 dataset_name="amazon_polarity",
                 batch_size=32,
                 n=6000,
                 prompt_fn=format_imdbs_multishot,
                ):
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.save_hyperparameters(ignore=["model", "tokenizer"])
        self.dataset = None
        self.prompt_fn=prompt_fn

    def setup(self, stage: str):
        
        # just setup once
        if self.dataset is not None:
            print('skipping setup, using cached values')
            return None

        self.dataset = load_dataset(self.hparams.dataset_name, split="test")

        # in ELK they cache as a huggingface dataset
        self.neg_hs, self.pos_hs, self.y, self.all_neg_ans, self.all_pos_ans = batch_hidden_states(
            self.model, self.tokenizer, self.dataset, self.prompt_fn, n=self.hparams.n, layers=[2, -2])

        # let's create a simple 50/50 train split (the data is already randomized)
        n = len(self.y)
        val_split = int(n * 0.5)
        test_split = int(n * 0.75)
        neg_hs_train, pos_hs_train, y_train = self.neg_hs[:
                                                     val_split], self.pos_hs[:
                                                                        val_split], self.y[:
                                                                                      val_split]
        neg_hs_val, pos_hs_val, y_val = self.neg_hs[val_split:test_split], self.pos_hs[
            val_split:test_split], self.y[val_split:test_split]
        neg_hs_test, pos_hs_test, y_test = self.neg_hs[test_split:],self. pos_hs[
            test_split:], self.y[test_split:]

        # for simplicity we can just take the difference between positive and negative hidden states
        # (concatenating also works fine)
        self.x_train = neg_hs_train - pos_hs_train
        self.x_val = neg_hs_val - pos_hs_val
        self.x_test = neg_hs_test - pos_hs_test

        # normalize
        self.scaler = RobustScaler()
        self.scaler.fit(self.x_train)
        self.x_train = self.scaler.transform(self.x_train)
        self.x_val = self.scaler.transform(self.x_val)
        self.x_test = self.scaler.transform(self.x_test)

        self.ds_train = TensorDataset(torch.from_numpy(neg_hs_train).float(),
                                      torch.from_numpy(pos_hs_train).float(),
                                      torch.from_numpy(y_train).float())

        self.ds_val = TensorDataset(torch.from_numpy(neg_hs_val).float(),
                                    torch.from_numpy(pos_hs_val).float(),
                                    torch.from_numpy(y_val).float())

        self.ds_test = TensorDataset(torch.from_numpy(neg_hs_test).float(),
                                     torch.from_numpy(pos_hs_test).float(),
                                     torch.from_numpy(y_test).float())

    def train_dataloader(self):
        return DataLoader(self.ds_train,
                          batch_size=self.hparams.batch_size,
                          shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.ds_val, batch_size=self.hparams.batch_size)

    def test_dataloader(self):
        return DataLoader(self.ds_test, batch_size=self.hparams.batch_size)



In [34]:
# test and cache
dm = imdbHSDataModule(model, tokenizer)
dm.setup('train')
dl = dm.val_dataloader()
b = next(iter(dl))
b

Found cached dataset amazon_polarity (/home/ubuntu/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/a27b32b7e7b88eb274a8fa8ba0f654f1fe998a87c22547557317793b5d2772dc)
Loading cached shuffled indices for dataset at /home/ubuntu/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/a27b32b7e7b88eb274a8fa8ba0f654f1fe998a87c22547557317793b5d2772dc/cache-0a5d0b47b5e8dfc6.arrow


get hidden states:   0%|          | 0/500 [00:00<?, ?it/s]

# LightningModel